In [1]:
# noexport

!typech efficacy_utils.ipynb

[NbConvertApp] Converting notebook efficacy_utils.ipynb to python
efficacy_utils.py:146: error: Name 'get_ipython' is not defined
efficacy_utils.py:150: error: Name 'get_ipython' is not defined
efficacy_utils.py:167: error: Name 'get_ipython' is not defined
efficacy_utils.py:203: error: Name 'condition_to_mlengths' is not defined


In [2]:
from retention_utils import *

In [3]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [4]:
#installs_with_experiment_vars = get_installs_with_experiment_vars()

In [5]:
#print(installs_with_experiment_vars[0])

In [6]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen'
  groups = ['survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [7]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

Created cache directory 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install


In [8]:
# noexport

for x in get_collection_names():
  if 'per_day' in x and 'visits' not in x:
    print(x)
    break

edceda6041ef668213a96ba5_synced:seconds_on_domain_per_day


In [9]:
@msgpackmemo1arg
def get_epoch_to_domain_to_time_spent(install):
  seconds_on_domain_per_day = get_collection_for_install(install, 'synced:seconds_on_domain_per_day')
  output = {}
  for x in seconds_on_domain_per_day:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_day')
      print(x)
      continue
    domain = x['key']
    epoch = x['key2']
    seconds = x['val']
    if epoch not in output:
      output[epoch] = {}
    output[epoch][domain] = seconds
  return output

Created cache directory 2019_06_06/get_epoch_to_domain_to_time_spent


In [11]:
# noexport

condition_to_installs = get_condition_to_installs_for_random_assignment_abtest()


In [12]:
# noexport

for condition,installs in condition_to_installs.items():
  for install in installs:
    print(sorted(get_epoch_to_domain_to_time_spent(install).keys()))
    break

exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/000a07dad34314af5a49d683.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/000a07dad34314af5a49d683.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/000a07dad34314af5a49d683.msgpack for arg 000a07dad34314af5a49d683
exception in msgpackmemo1arg for file 2019_06_06/get_collection_items/f42309bc714b8c7bdc9f6bd6_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_collection_items/f42309bc714b8c7bdc9f6bd6_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_06_06/get_collection_items/f42309bc714b8c7bdc9f6bd6_synced:seconds_on_domain_per_day.msgpack for arg f42309bc714b8c7bdc9f6bd6_synced:seconds_on_domain_per_day
done with computation 2019_06_06/get_collection_items/f42309bc714b8c7bdc9f6bd6_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/000a07dad34314af5a

In [15]:
import math

def make_domain_to_daily_time_dataframe():
  output = []
  for condition,installs in condition_to_installs.items():
    #condition_to_lengths[condition] = []
    for install in installs:
      for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
        total_time_spent = sum(domain_to_time_spent.values())
        output.append({
          'user': install,
          'epoch': epoch,
          'time': total_time_spent,
          'logtime': math.log(total_time_spent),
          'condition': condition,
        })
#         for domain,total_time_spent in  domain_to_time_spent.items():
#           output.append({
#             'domain': domain,
#             'user': install,
#             'epoch': epoch,
#             'time': total_time_spent,
#             'logtime': math.log(total_time_spent),
#             'condition': condition,
#           })
  return to_dataframe(output)

In [16]:
df = make_domain_to_daily_time_dataframe()

exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/02d0650dd54dad3d48403891.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/02d0650dd54dad3d48403891.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/02d0650dd54dad3d48403891.msgpack for arg 02d0650dd54dad3d48403891
exception in msgpackmemo1arg for file 2019_06_06/get_collection_items/afe536d6907aedb612edf18b_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_collection_items/afe536d6907aedb612edf18b_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_06_06/get_collection_items/afe536d6907aedb612edf18b_synced:seconds_on_domain_per_day.msgpack for arg afe536d6907aedb612edf18b_synced:seconds_on_domain_per_day
done with computation 2019_06_06/get_collection_items/afe536d6907aedb612edf18b_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/02d0650dd54dad3d48

done with computation 2019_06_06/get_collection_items/702c442bd2d6b57da314029a_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/0b8125feba367e2c37fd3149.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/0f04d202bb1ac444d0f73250.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/0f04d202bb1ac444d0f73250.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/0f04d202bb1ac444d0f73250.msgpack for arg 0f04d202bb1ac444d0f73250
exception in msgpackmemo1arg for file 2019_06_06/get_collection_items/93f5b666051403f6dedc9490_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_collection_items/93f5b666051403f6dedc9490_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_06_06/get_collection_items/93f5b666051403f6dedc9490_synced:seconds_on_domain_per_day.msgpack for arg 93f5b666051403f6dedc9490_synced:seconds_on_

done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/1f844761e55e04d25b7bf766.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/2163580c919e16533b4857cb.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/2163580c919e16533b4857cb.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/2163580c919e16533b4857cb.msgpack for arg 2163580c919e16533b4857cb
exception in msgpackmemo1arg for file 2019_06_06/get_collection_items/4e0bf4a415c346a14d7641e0_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_collection_items/4e0bf4a415c346a14d7641e0_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_06_06/get_collection_items/4e0bf4a415c346a14d7641e0_synced:seconds_on_domain_per_day.msgpack for arg 4e0bf4a415c346a14d7641e0_synced:seconds_on_domain_per_day
done with computation 2019_06_06/get_collection_items/4e0bf4a415c346a14d7641e0_synced:seconds_on_domain_p

done with computation 2019_06_06/get_collection_items/b5654f75537908c7365e5005_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/3d347287b545aae030c3d4b9.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/40ae4630199bff5a2c454cbd.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/40ae4630199bff5a2c454cbd.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/40ae4630199bff5a2c454cbd.msgpack for arg 40ae4630199bff5a2c454cbd
exception in msgpackmemo1arg for file 2019_06_06/get_collection_items/769b4a8de599c811ca3c675e_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_collection_items/769b4a8de599c811ca3c675e_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_06_06/get_collection_items/769b4a8de599c811ca3c675e_synced:seconds_on_domain_per_day.msgpack for arg 769b4a8de599c811ca3c675e_synced:seconds_on_

done with computation 2019_06_06/get_collection_items/af63777e3ee57c0cece80a0d_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/5758c8a3472b56dad5035bdf.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/586b6e538ffc9b50d4211cd5.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/586b6e538ffc9b50d4211cd5.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/586b6e538ffc9b50d4211cd5.msgpack for arg 586b6e538ffc9b50d4211cd5
exception in msgpackmemo1arg for file 2019_06_06/get_collection_items/bd9b64337a7cc82318b08d52_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_collection_items/bd9b64337a7cc82318b08d52_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_06_06/get_collection_items/bd9b64337a7cc82318b08d52_synced:seconds_on_domain_per_day.msgpack for arg bd9b64337a7cc82318b08d52_synced:seconds_on_

...

done with computation 2019_06_06/get_collection_items/cd35f84b81596b0930f8d9b5_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/450950ad4c511a3c34845dd3.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/47519674af7206934615e68f.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/47519674af7206934615e68f.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/47519674af7206934615e68f.msgpack for arg 47519674af7206934615e68f
exception in msgpackmemo1arg for file 2019_06_06/get_collection_items/99a79ee2a6de6c8404f66a79_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_collection_items/99a79ee2a6de6c8404f66a79_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_06_06/get_collection_items/99a79ee2a6de6c8404f66a79_synced:seconds_on_domain_per_day.msgpack for arg 99a79ee2a6de6c8404f66a79_synced:seconds_on_

done with computation 2019_06_06/get_collection_items/3bd9b4d9398fb06bd0bad361_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/7514f3ab27efba31b4fba12e.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/765480134f7ae69ba2b44dda.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/765480134f7ae69ba2b44dda.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/765480134f7ae69ba2b44dda.msgpack for arg 765480134f7ae69ba2b44dda
exception in msgpackmemo1arg for file 2019_06_06/get_collection_items/225093d6a7c77482567dd09c_synced:seconds_on_domain_per_day.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_collection_items/225093d6a7c77482567dd09c_synced:seconds_on_domain_per_day.msgpack'
performing computation 2019_06_06/get_collection_items/225093d6a7c77482567dd09c_synced:seconds_on_domain_per_day.msgpack for arg 225093d6a7c77482567dd09c_synced:seconds_on_

done with computation 2019_06_06/get_collection_items/bc2a2a9aa64af2b5cd1b403a_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/9737a62cb2a23cceb50348ef.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/97cc4190e861491bf58b094e.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/97cc4190e861491bf58b094e.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/97cc4190e861491bf58b094e.msgpack for arg 97cc4190e861491bf58b094e
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/97cc4190e861491bf58b094e.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/998eeff51efe69534f15fd44.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/998eeff51efe69534f15fd44.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/998eeff51efe69534f15fd44.msgpack for arg 998

done with computation 2019_06_06/get_collection_items/6299956f1fee8b164bfecd1f_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/c6d38f5f5f9ed13d67f2321d.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/c7a546926bf5809128ab53a0.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/c7a546926bf5809128ab53a0.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/c7a546926bf5809128ab53a0.msgpack for arg c7a546926bf5809128ab53a0
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/c7a546926bf5809128ab53a0.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/c85af79c93950e7e37409423.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/c85af79c93950e7e37409423.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/c85af79c93950e7e37409423.msgpack for arg c85

done with computation 2019_06_06/get_collection_items/445917bd56af87d1efe704f9_synced:seconds_on_domain_per_day.msgpack
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/eb80addcba2e2d9a09c2d9e2.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/eb85cd63973a61d63ed9840b.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/eb85cd63973a61d63ed9840b.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/eb85cd63973a61d63ed9840b.msgpack for arg eb85cd63973a61d63ed9840b
done with computation 2019_06_06/get_epoch_to_domain_to_time_spent/eb85cd63973a61d63ed9840b.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_epoch_to_domain_to_time_spent/ed1cd8ffdb099cb63de09d12.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_epoch_to_domain_to_time_spent/ed1cd8ffdb099cb63de09d12.msgpack'
performing computation 2019_06_06/get_epoch_to_domain_to_time_spent/ed1cd8ffdb099cb63de09d12.msgpack for arg ed1

In [17]:
#print(df)

In [18]:
%Rpush df

/usr/local/lib/python3.7/dist-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning:

from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.



In [19]:
%%R
library(lme4)
#library(sjPlot)
library(lmerTest)
#library(stargazer)


/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

载入需要的程辑包：Matrix


/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:


载入程辑包：‘lmerTest’



/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

The following object is masked from ‘package:lme4’:

    lmer



/usr/local/lib/python3.7/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:

The following object is masked from ‘package:stats’:

    step





In [20]:
# %%R

# df$user <- as.factor(df$user)
# #df$domain <- as.factor(df$domain)
# df$condition <- as.factor(df$condition)
# df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
# df$epoch <- as.factor(df$epoch)
# df$logtime <- as.numeric(df$logtime)
# df$time <- as.numeric(df$time)
# summary(df)



In [21]:
%%R

df$user <- as.factor(df$user)
#df$domain <- as.factor(df$domain)
df$condition <- as.factor(df$condition)
df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
df$epoch <- as.factor(df$epoch)
df$logtime <- as.numeric(df$logtime)
df$time <- as.numeric(df$time)
summary(df)


#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
#show(results)
#show(summary(results))
#class(results) <- "lmerMod"
#stargazer(results)

                       user           epoch            time      
 59a681645c83e77999092aa7:  272   1075   :  182   Min.   :    1  
 e8d4be7c60191c95048a950b:  233   1068   :  174   1st Qu.: 1879  
 4ed86bc56c8a5e831bb48afc:  229   1070   :  174   Median : 5503  
 1b1ee7527ee7195d80e732ae:  228   1077   :  173   Mean   : 8055  
 f76f3d99da9e92d598d673f0:  226   1069   :  172   3rd Qu.:10961  
 5867a31094f50f539ab6c65a:  224   1074   :  170   Max.   :86402  
 (Other)                 :22357   (Other):22724                  
    logtime                         condition   
 Min.   : 0.000   survey_nochoice_nothing:7416  
 1st Qu.: 7.538   survey_nochoice_easy   :6369  
 Median : 8.613   survey_nochoice_medium :5629  
 Mean   : 8.209   survey_nochoice_hard   :4355  
 3rd Qu.: 9.302                                 
 Max.   :11.367                                 
                                                


In [22]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      condition_to_lengths[condition].append(total_time_spent)
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [23]:
# noexport
# bad


condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for domain,intervention_to_session_lengths in get_domain_to_intervention_to_session_lengths_for_install(install).items():
      if domain != 'www.youtube.com':
        continue
      for intervention,session_lengths in intervention_to_session_lengths.items():
        for session_length in session_lengths:
          condition_to_lengths[condition].append(session_length)


exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/000a07dad34314af5a49d683.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/000a07dad34314af5a49d683.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/000a07dad34314af5a49d683.msgpack for arg 000a07dad34314af5a49d683
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/000a07dad34314af5a49d683.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/02d0650dd54dad3d48403891.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/02d0650dd54dad3d48403891.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/02d0650dd54dad3d48403891.msgpack for arg 02d0650dd54dad3d48403891
done with computation 2019_06_

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/1b1ee7527ee7195d80e732ae.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/1f272e6550e639e267e54686.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/1f272e6550e639e267e54686.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/1f272e6550e639e267e54686.msgpack for arg 1f272e6550e639e267e54686
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/1f272e6550e639e267e54686.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/1f844761e55e04d25b7bf766.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/1f844761e55e04d25b7bf766.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/46e5f0f87f4ab3042eb780e4.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/472d641f555a17e6c02aa050.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/472d641f555a17e6c02aa050.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/472d641f555a17e6c02aa050.msgpack for arg 472d641f555a17e6c02aa050
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/472d641f555a17e6c02aa050.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/4820121c61d7c5b40ee8214a.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/4820121c61d7c5b40ee8214a.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/68fccbb8ce1e6ef99b9e600b.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/6a7adbc82559f660886b2cea.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/6a7adbc82559f660886b2cea.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/6a7adbc82559f660886b2cea.msgpack for arg 6a7adbc82559f660886b2cea
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/6a7adbc82559f660886b2cea.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/7239b3fcaf2f678058cf4fc1.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/7239b3fcaf2f678058cf4fc1.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/94b07f354d026941f40d3158.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/980d3fbc5d1b97caca932c6c.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/980d3fbc5d1b97caca932c6c.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/980d3fbc5d1b97caca932c6c.msgpack for arg 980d3fbc5d1b97caca932c6c
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/980d3fbc5d1b97caca932c6c.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/9abdb145d6e49e993045be95.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/9abdb145d6e49e993045be95.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

...

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/3c163c844eed7cf2acb8980f.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/44298e62fc8cf3737c917664.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/44298e62fc8cf3737c917664.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/44298e62fc8cf3737c917664.msgpack for arg 44298e62fc8cf3737c917664
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/44298e62fc8cf3737c917664.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/444e73cf4a980e1466894756.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/444e73cf4a980e1466894756.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/7514f3ab27efba31b4fba12e.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/765480134f7ae69ba2b44dda.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/765480134f7ae69ba2b44dda.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/765480134f7ae69ba2b44dda.msgpack for arg 765480134f7ae69ba2b44dda
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/765480134f7ae69ba2b44dda.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/7a994ccad67df2369e53541a.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/7a994ccad67df2369e53541a.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/a53d08a935524c8aad8bead2.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/a66d765488816e717668e3e0.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/a66d765488816e717668e3e0.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/a66d765488816e717668e3e0.msgpack for arg a66d765488816e717668e3e0
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/a66d765488816e717668e3e0.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/a79763041b4e8de196fe10ed.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/a79763041b4e8de196fe10ed.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/cb5bc69d6cec8a6952b78e25.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/cb9d847e4243f6c01c565b7c.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/cb9d847e4243f6c01c565b7c.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/cb9d847e4243f6c01c565b7c.msgpack for arg cb9d847e4243f6c01c565b7c
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/cb9d847e4243f6c01c565b7c.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/cbe1aa56a88443de255aac17.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/cbe1aa56a88443de255aac17.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/f250f47a6f12d8f60d12f363.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/f57e024720729659b8a4e908.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/f57e024720729659b8a4e908.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/f57e024720729659b8a4e908.msgpack for arg f57e024720729659b8a4e908
done with computation 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/f57e024720729659b8a4e908.msgpack
exception in msgpackmemo1arg for file 2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/f8d65f724e90a013f958e1ad.msgpack
[Errno 2] 没有那个文件或目录: '2019_06_06/get_domain_to_intervention_to_session_lengths_for_install/f8d65f724e90a013f958e1ad.msgpack'
performing computation 2019_06_06/get_domain_to_intervention_to_

In [24]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    all_lengths_for_user = []
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      all_lengths_for_user.append(total_time_spent)
    if len(all_lengths_for_user) == 0:
      continue
    condition_to_lengths[condition].append(np.median(all_lengths_for_user))
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [25]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      if 'www.facebook.com' not in domain_to_time_spent:
        continue
      time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      condition_to_lengths[condition].append(time_on_facebook)

In [26]:
# noexport
from scipy.stats import ttest_ind

condition_to_mlengths = {}
for condition,session_lengths in condition_to_lengths.items():
  print(condition, np.median(session_lengths))
  condition_to_mlengths[condition] = np.median(session_lengths) / 60

print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_nothing']))
#print(ttest_ind(condition_to_lengths['survey_nochoice_easy'], condition_to_lengths['survey_nochoice_medium']))


survey_nochoice_medium 397.0
survey_nochoice_nothing 505.0
survey_nochoice_easy 334.0
survey_nochoice_hard 382.0
Ttest_indResult(statistic=-2.4014372908193136, pvalue=0.016362103909610175)


In [31]:
from plot_utils import *


In [32]:
#from plot_utils import *
plot_dict_as_bar(condition_to_mlengths, title='Total minutes spent daily on sites', xlabel='Difficulty', ylabel='Minutes spent daily on sites', remap_labels={'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard'})

In [29]:
# for install in installs_with_experiment_vars:
#   #seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
#   #interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
#   res = get_domain_to_intervention_to_session_lengths_for_install(install)
#   #if len(res) > 0:
#   #  print(res)
#   #  break